## MNIST classification

In this notebook we tackle the perhaps most well known problem in all of machine learning, classifying hand-written digits.

The particular dataset we will use is the MNIST (Modified National Institute of Standards and Technology)
The digits are 28x28 pixel images that look somewhat like this:

![](https://user-images.githubusercontent.com/2202312/32365318-b0ccc44a-c079-11e7-8fb1-6b1566c0bdc4.png)

Each digit has been hand classified, e.g. for the above 9-7-0-9-0-...

Our task is to teach a machine to perform this classification, i.e. we want to find a function $\mathcal{T}_\theta$ such that

| | |
|-|-|
|$\mathcal{T}_\theta$(|<img align="center" src="https://user-images.githubusercontent.com/2202312/33177374-b134e572-d062-11e7-87c7-0574c6f5bee9.png" width="28"/>|) = 4|

# Import dependencies

This should run without errors if all dependencies are installed properly.

In [ ]:
%matplotlib notebook

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
# Start a tensorflow session
session = tf.InteractiveSession()

# Set the random seed to enable reproducible code
np.random.seed(0)

# Get data and utilities

We now need to get the data we will use, which in this case is the famous [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, a set of digits 70000 hand-written digits, of which 60000 are used for training and 10000 for testing.

In addition to this, we create a utility `evaluate(...)` that we will use to evaluate how good the classification is.

Get MNIST data

In [ ]:
mnist = input_data.read_data_sets('MNIST_data')

In [ ]:
import matplotlib.cm

In [ ]:
plt.imshow(mnist.test.images[0].reshape(28,-1),cmap='Greys_r')

Read the 10000 mnist test points

In [ ]:
batch = mnist.test.next_batch(10000)
test_images = batch[0].reshape([-1, 28, 28, 1])
test_labels = batch[1]

def evaluate(result_tensor, data_placeholder):
    """Evaluate a reconstruction method.

    Parameters
    ----------
    result_tensor : `tf.Tensor`, shape (None,)
        The tensorflow tensor containing the result of the classification.
    data_placeholder : `tf.Tensor`, shape (None, 28, 28, 1) or (None, 784)
        The tensorflow tensor containing the input to the classification operator.

    Returns
    -------
    MSE : float
        Mean squared error of the reconstruction.
    """
    feed_images = np.reshape(test_images, [-1, *data_placeholder.shape[1:]])
    result = result_tensor.eval(
        feed_dict={data_placeholder: feed_images})

    return np.mean(result == test_labels)

Create placeholders. Placeholders are needed in tensorflow since tensorflow is a lazy language,
and hence we first define the computational graph with placeholders as input, and later we evaluate it.

In [ ]:
with tf.name_scope('placeholders'):
    images = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
    true_labels = tf.placeholder(tf.int32, shape=[None])

# Logistic regression


We start with [logistic regression](https://en.wikipedia.org/wiki/Logistic_regression), perhaps the most well known and widely applied classification method.

The first problem we need to solve is that the values we try to regress against are discrete (e.g. [0, 1, 2, ..., 9]) which does not work very well with continuous optimization. To solve this we convert the values to a one-hot encoding, embedding the values into $\mathbb{R}^{10}$:



In [ ]:
toh = tf.one_hot([0, 1, 2], depth=3)
toh.eval()

This can be seen as a probabilistic encoding, i.e. we can estimate that a number is 10% 1 and 90% 2. For our training data, we have 100% certanity for each digit. 

The estimator used for logistic regression is

$$
p_x(\text{label=$i$}) = \frac{\exp(\langle w_i, x \rangle + b_i)}{\sum_{j=0}^9 \exp(\langle w_j, x \rangle + b_j)}
$$

Here, $p_x(\text{label=$i$})$ is the probability of an image $x$ belonging to a category $i$, $w_i \in \mathbb{R}^{28 \times 28}$ and $b_i \in \mathbb{R}$

The loss function is a comparison between the probability distribution $p_x$ and the deterministic probability distribution $q_x$.
We use the *cross entropy* for this. The loss function for each image is:
\\[
-\sum_{i=0}^9 q_x(\text{label=$i$}) \ln(p_x(\text{label=$i$}))
\\]

The final loss function is the mean value of the cross entropy (implicitly assuming that the images are uniformly distributed):
\\[
L(p) := -\frac{1}{N}\sum_{x=1}^N\sum_{i=0}^9 q_x(\text{label=$i$})\ln(p_x(\text{label=$i$}))
\\]

### Elementary Implementation

We start with an elementary implementation in `TensorFlow`.

In [ ]:
with tf.name_scope('elementary'):
    X = tf.placeholder(shape=(None, 784), dtype=tf.float32, name="X")
    weights = tf.Variable(tf.random_normal((784, 10)), name="weights")
    bias = tf.Variable(tf.zeros((10)), name="bias")
    lin = tf.matmul(X, weights)
    lin_ = lin + bias
    elin_ = tf.exp(lin_)
    Z = tf.reduce_sum(tf.exp(lin_), axis=1, keep_dims=True)
    prob = elin_ / Z
    log_prob = tf.log(prob)

In [ ]:
with tf.name_scope("elementary_loss"):
    labels = tf.placeholder(shape=(None,), dtype=tf.int32)
    determ = tf.one_hot(labels, depth=10)
    loss = -tf.reduce_mean(determ*log_prob)

In [ ]:
with tf.name_scope("elementary_training"):    
    learning_rate = .1
    batch_size = 2**7

    variables = [weights, bias]
    gradients = tf.gradients(loss, variables)
    update_ops = [var.assign(var - learning_rate*grad) 
                  for var, grad in zip(variables, gradients)]

In [ ]:
init = tf.global_variables_initializer().run()

In [ ]:
feed_dict={labels:mnist.train.labels[:batch_size], X:mnist.train.images[:batch_size]}
for i in range(100000):
    images_, labels_ = mnist.train.next_batch(batch_size)
    session.run(update_ops, 
                feed_dict={labels:labels_, X:images_})
    
    if i % 1000 == 0:
        print("{:.1f}%, ".format(evaluate(tf.argmax(log_prob, axis=1), X)*100), end="")

### Using TensorFlow libraries

In [ ]:
with tf.name_scope('logistic_regression'):
    x = tf.contrib.layers.flatten(images)
    logits = tf.contrib.layers.fully_connected(x, 10,
                                               activation_fn=None)
    pred = tf.argmax(logits, axis=1)
    
with tf.name_scope('optimizer'):
    one_hot_labels = tf.one_hot(true_labels, depth=10)
    
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_labels,
                                                   logits=logits)
    optimizer = tf.train.AdamOptimizer().minimize(loss)

# Initialize all TF variables
session.run(tf.global_variables_initializer())

for i in range(10000):
    batch = mnist.train.next_batch(128)
    train_images = batch[0].reshape([-1, 28, 28, 1])
    train_labels = batch[1]

    session.run(optimizer, feed_dict={images: train_images, 
                                      true_labels: train_labels})

    if i % 100 == 0:
        print('{} Average correct: {}'.format(
                i, evaluate(pred, images)))

# Multilayer Perceptron

The first "deep" neural networks were [multilayer perceptrons](https://en.wikipedia.org/wiki/Multilayer_perceptron), in these we have a function of the following form

$$
\rho(W_3\rho(W_2\rho(W_1 x + b_1) + b_2) + b_3)
$$

Where $W_i$ are matrices and $b_i$ vectors. Note that the logistic regression can be cast into this form (how?).

In [ ]:
with tf.name_scope('logistic_regression'):
    x = tf.contrib.layers.flatten(images)
    x = tf.contrib.layers.fully_connected(x, 128)  # the default activation function is ReLU
    x = tf.contrib.layers.fully_connected(x, 32)
    logits = tf.contrib.layers.fully_connected(x, 10,
                                               activation_fn=None)
    pred = tf.argmax(logits, axis=1)
    
with tf.name_scope('optimizer'):
    one_hot_labels = tf.one_hot(true_labels, depth=10)
    
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_labels,
                                                   logits=logits)
    optimizer = tf.train.AdamOptimizer().minimize(loss)

# Initialize all TF variables
session.run(tf.global_variables_initializer())

for i in range(10000):
    batch = mnist.train.next_batch(128)
    train_images = batch[0].reshape([-1, 28, 28, 1])
    train_labels = batch[1]

    session.run(optimizer, feed_dict={images: train_images, 
                                      true_labels: train_labels})

    if i % 100 == 0:
        print('{} Average correct: {}'.format(
                i, evaluate(pred, images)))

# Convolutional network

Convolutional neural networks are a corner-stone of the deep learning revolution. Here instead of using traditionall fully-connected layers which connect each point with all other points, we use spatial convolutions instead. By doing this, we get a translation invariant operator that acts locally. In order to get non-local behaviour we stack several of these on top of each other.

The following code is a very simplified convolutional neural network for digit classification:

In [ ]:
with tf.name_scope('convolutional_network'):
    x = tf.contrib.layers.conv2d(images, num_outputs=32, kernel_size=3, stride=2)
    x = tf.contrib.layers.conv2d(x, num_outputs=32, kernel_size=3, stride=2)
    x = tf.contrib.layers.flatten(x)
    
    x = tf.contrib.layers.fully_connected(x, 128)
    logits = tf.contrib.layers.fully_connected(x, 10,
                                               activation_fn=None)
    pred = tf.argmax(logits, axis=1)
    
with tf.name_scope('optimizer'):
    one_hot_labels = tf.one_hot(true_labels, depth=10)
    
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_labels,
                                                   logits=logits)
    optimizer = tf.train.AdamOptimizer().minimize(loss)

# Initialize all TF variables
session.run(tf.global_variables_initializer())

for i in range(10000):
    batch = mnist.train.next_batch(128)
    train_images = batch[0].reshape([-1, 28, 28, 1])
    train_labels = batch[1]

    session.run(optimizer, feed_dict={images: train_images, 
                                      true_labels: train_labels})

    if i % 100 == 0:
        print('{} Average correct: {}'.format(
                i, evaluate(pred, images)))